##### Copyright 2020 Roland Mostoha

# Enhance - DeepLearning

Enhance is Tensorflow based regression project. Its primary goal is to make predictions based on your personal fitness goals using your health records.

In [ ]:
from datetime import date
from datetime import timedelta

import fitbit as fitbit
import pandas as pd
import tensorflow as tf
from fitbit import Fitbit

from auth.fitbit_authenticator import FitbitAuthenticator
from data.data_loader import DataLoader
from data.dataset_generator import DatasetGenerator
from data.model.records import KEYS_ALL_HEALTH_RECORDS
from data.provider.fitbit_data_provider import FitbitDataProvider
from plotter import plotter
from trainer.dnn_trainer import DNNTrainer

## 1. Fitbit Authentication

You can skip this step if you don't want to use Fitbit provider.

In [ ]:
auth = FitbitAuthenticator("../configs/fitbit_auth_config.json")
auth.authenticate_in_browser()

fitbit = fitbit.Fitbit(auth.client_id, auth.client_secret,
                       access_token=auth.access_token,
                       refresh_token=auth.refresh_token,
                       system=Fitbit.METRIC)

## 2. Load and generate health records

Set start date and end date from which period you want to collect the data.

In [ ]:
start_date = date(year=2020, month=1, day=1)
end_date = date.today() - timedelta(days=1)

Defining the data provider, loading and generating records. You can also use `RandomDataProvider` or a custom one.

In [ ]:
data_provider = FitbitDataProvider(fitbit, start_date, end_date)

loader = DataLoader(data_provider)
loader.generate_records()

Plot the records for manual evaluation.

In [ ]:
plotter.plot_all_records(loader.records)

Writing health records to csv.

In [ ]:
loader.write_to_csv("../data/raw/health_records.csv")

## 3. Generate dataset

Generating the dataset for the model by the collected health records

In [ ]:
pd.set_option('display.max_columns', 15)

data_frame = pd.read_csv("../data/raw/health_records.csv")
data_frame = data_frame.fillna(data_frame.mean())
data_frame = data_frame.drop(columns=['record_date'])

# Check dataset stats
print(data_frame.describe().transpose())

Check the most recent health record and define your goal

In [ ]:
last_record = data_frame.tail(1)
print('Most recent health record:\n', last_record)

# Set the modified values for my personal goal
my_goal_changes = {'weight': 65}

# Merging the most recent record with the modified values
my_goal = last_record.copy()
for key, value in my_goal_changes.items():
    my_goal[key] = value
print('My goal merged with most recent health record:\n', my_goal)

record_keys = KEYS_ALL_HEALTH_RECORDS

# Skip the modified keys, there is no need to predict overwritten records
for key in my_goal_changes.keys():
    record_keys.remove(key)

## 4. Train and run the models

You can customize the trainers or write a new one besides the built-in ones: `LinearRegressionTrainer`, `DNNTrainer`

In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(60)

prediction_results = 'Results for my goal {}'.format(my_goal_changes)

for key in record_keys:
    dataset = DatasetGenerator(data_frame, target_feature=key)
    dataset.split(train_ratio=0.75, valid_ratio=0.15, test_ratio=0.10)

    trainer = DNNTrainer(dataset, key)

    model = trainer.train()

    # Drop the target feature that we are train for
    my_goal_input = my_goal.drop(columns=[key])

    prediction = model.predict(my_goal_input)

    feature_prediction = 'Predicted {} value for my goal: {}'.format(key, prediction)
    prediction_results += '\n' + feature_prediction

print(prediction_results)